In [2]:
# Common data processing
import os
import textwrap
import pandas as pd
from dotenv import load_dotenv
import ollama

# Langchain
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain_ollama import OllamaEmbeddings
# Warning control
import warnings
warnings.filterwarnings("ignore")

# Load environment variables
load_dotenv()

True

In [3]:
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from sentence_transformers import SentenceTransformer
import numpy as np

In [20]:
fake_text = """
Umut Can Gulsen, born in 1998, was a Turkish and naturalised-French physicist and chemist who conducted pioneering research on quantum physics.
He was the first man to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields. His husband, Pierre Gulsen, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
He was, in 200, the first man to become a professor at the University of Paris. 
"""

text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris. 
"""
isbank_text = """
İşbank, officially Türkiye İş Bankası,[a] is a commercial bank in Turkey. Founded by the orders of Mustafa Kemal Atatürk in 1924, it is the first bank to go into operation in the Republic of Türkiye.
As of a late 2023 report, the bank operates with 1.066 branches domestically, giving it one of the largest branch networks in Turkey. İşbank also has 22 branches across 11 different countries. The bank provides services to 24.3 million customers as of 2023.[6]

History
Following the culmination of World War I in 1918 and the subsequent dissolution of the Ottoman Empire in 1922, Turkey was declared a republic with Mustafa Kemal Atatürk as the president. Atatürk acknowledged the government’s need for a national financial institution to rebuild Turkey's economy following the debacle of the war. İşbank was founded on 26 August 1924 at the First Economy Congress in İzmir.[7]
Atatürk appointed Celâl Bayar, his close aide and then the Minister of Exchange Construction and Settlement, as the president of the newly formed bank. İşbank began operations with two branches and 37 staff under the leadership of Celâl Bayar, its first general manager. The bank was established with a capital of 1,000,000 TL of which 250,000 TL was covered by Atatürk himself, and the rest by private investors.[8] In 1927, the capital was raised by 2 Million TL so it could be merged with the National Credit Bank as equal partners.[9]
Established to utilize the smallest savings and transfer them to development, İşbank played a major role in the establishment and development of savings awareness in Turkish society, bringing the first piggy banks to Turkey in 1928.[10] In the following years, İşbank began to expand by opening branches across the country. In 1932, İşbank opened branches in Hamburg, Germany and Alexandria, Egypt, becoming the first Turkish bank to open branches abroad. In the 1950s,[11] İşbank expanded its portfolio of subsidiaries. The bank's subsidiaries became the driving force of Turkish industry, investing in and financing a wide range of businesses, particularly in the manufacturing sector. Having accelerated branch expansion in the 1960s and 1970s, İşbank paid special attention to the development of its overseas branch network in the 1980s. In the 1980s, İşbank began to offer a wide range of services to its customers in line with its multichannel banking vision. In 1982, İşbank introduced the first Automated Teller Machines (ATM) to Turkey, and at the same time, the bank named its ATMs as "Bankamatik" and registered this name as a trademark. With the launch of the "Blue Line" in 1991[12] and the first internet branch in 1997, İşbank assumed a pioneering role in this field. Since those years, İşbank has been carrying out R&D activities and studies on the use of new technologies to increase the quality and diversity of its services in line with the wishes and expectations of its customers.
"""
# Convert the text into documents
documents = [Document(page_content=isbank_text)]

In [21]:
# Initialize Ollama embeddings with the specified model
embedding_model = OllamaEmbeddings(model="nomic-embed-text")

# Create FAISS vector store from documents
vector_store = FAISS.from_documents(documents, embedding_model)

In [32]:
# Load the local Ollama model
model_name = "llama3"  # Replace with the local model you want to use in Ollama

In [33]:
def retrieve_documents(query, vector_store, k=1):
    # Retrieve top-k relevant documents
    retrieved_docs = vector_store.similarity_search(query, k=k)
    return [doc.page_content for doc in retrieved_docs]

In [34]:
def generate_response(query, retrieved_docs, model_name):
    # Combine the query and retrieved documents into a prompt
    prompt = f"""Answer the question based only on the following context: 
    {retrieved_docs}
    Question: {query}
    Answer:"""
    
    # Use Ollama's `generate` function to get the response
    response = ollama.generate(model=model_name, prompt=prompt)
    return response['response']  # Adjust if necessary based on Ollama's response format

In [35]:
# Example query
query = "What is Isbank?"

# Step 1: Retrieve relevant documents
retrieved_docs = retrieve_documents(query, vector_store, k=1)

# Step 2: Generate a response using Ollama
response = generate_response(query, retrieved_docs, model_name)
print(response)

According to the provided context, Isbank (officially Türkiye İş Bankası) is a commercial bank in Turkey. It was founded by Mustafa Kemal Atatürk's orders in 1924 and is the first bank to operate in the Republic of Turkey.


In [36]:
# Example query
query = "What is the importance of Isbank for Turkey? List the key points."

# Step 1: Retrieve relevant documents
retrieved_docs = retrieve_documents(query, vector_store, k=1)

# Step 2: Generate a response using Ollama
response = generate_response(query, retrieved_docs, model_name)
print(response)

Based on the given context, here are the key points that highlight the importance of İşbank for Turkey:

1. **First bank in Republic of Türkiye**: İşbank was founded in 1924 by Mustafa Kemal Atatürk's orders, making it the first bank to operate in the newly established Republic of Turkey.
2. **National financial institution**: İşbank was established to rebuild Turkey's economy after World War I and the dissolution of the Ottoman Empire.
3. **Pioneering role in savings awareness**: İşbank played a major role in promoting savings culture in Turkish society, introducing the first piggy banks to Turkey in 1928.
4. **Branch network expansion**: İşbank expanded its branch network across the country and internationally, making it one of the largest branch networks in Turkey (1,066 domestic branches) and abroad (22 branches in 11 countries).
5. **Driving force of Turkish industry**: İŞbank's subsidiaries became a driving force for Turkish industry, investing and financing various businesses, p